In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import typing as t
from arcgis.features import GeoAccessor, FeatureLayer, FeatureLayerCollection
from arcgis.gis import GIS
import arcpy
pgis = GIS("pro")
from pathlib import Path
import os

In [2]:
hf_ID = '65bd4723a3444a28adf20299799138c8'
fs_hf = pgis.content.get(hf_ID)
fl_hf = fs_hf.layers[0]
fl_features = fl_hf.query().features

In [3]:
#fl_hf.manager.overwrite() # = FeatureLayer(fl_hf)
#fl_features
#sdf_hf = pd.DataFrame.spatial.from_layer(fl_hf)
#sdf_hf
fset = fl_hf.query()
fset.features
#fset.sdf
#fl_hf.properties.capabilities
#fl_hf.edit_features(adds=)

[{"geometry": {"paths": [[[617316.3783, 6664268.0802], [617372.2023, 6664186.6995]]], "spatialReference": {"wkid": 3044, "latestWkid": 3044, "vcsWkid": 5941, "latestVcsWkid": 5941}}, "attributes": {"OBJECTID": 1, "Entity": "LWPolyline", "Handle": "2539", "Layer": "_Profiler", "LyrFrzn": 0, "LyrLock": 0, "LyrOn": 1, "LyrVPFrzn": 0, "LyrHandle": "1DF6", "Color": 7, "EntColor": -1, "LyrColor": 7, "BlkColor": 0, "Linetype": "Continuous", "EntLinetype": "Continuous", "LyrLnType": "Continuous", "BlkLinetype": "", "Elevation": 0, "Thickness": 0, "LineWt": 25, "EntLineWt": -1, "LyrLineWt": -3, "BlkLineWt": -1, "RefName": "", "LTScale": 1, "ExtX": 0, "ExtY": 0, "ExtZ": 1, "DocName": "Profiler_Samlet.dwg", "DocPath": "\\\\nsv2-nasuni-01\\Prosjekt\\O10244\\10244558-01\\10244558-01-03 ARBEIDSOMRAADE\\10244558-01 RIG\\10244558-01-04 TEGNINGER\\Profiler_Samlet.dwg", "DocType": "DWG", "DocVer": "AC1032", "DocUpdate": 1662731009000, "DocId": 3.34136406151006e+17, "Profil_Sone": "", "Profil_L\u00f8sneo

In [4]:
def add_picture(input_fc, inputField, pathField):
    # The input feature class must first be GDB attachments enabled
    arcpy.EnableAttachments_management(input_fc)
    # Use the match table with the Add Attachments tool
    arcpy.AddAttachments_management(input_fc, inputField, input_fc, inputField, 
                                    pathField, None)


def remove_word(sentence, word):
    """
        Input:
            sentence: string
            word: word to remove
        Output:
            returns sentence without word
    """
    if isinstance(sentence, str):
        return sentence.replace(word, '').strip()
    else:
        return None


def tolket(row):
    val = ''
    if isinstance(row['Metode'], str):        
        if 'Tolk' in row['Metode']:
            val = 'Tolk'
    return val


def bergkote(row):
    if row['Stopp'] in [93,94]:
        val = str(round(row['Z'] - row['Løsm'], 2))
    else:
        val = '~'
    return val


def z_kom(row):
    if row['Z'] == 0:
        val = 'Missing Z'
    else:
        val = ''
    return val


def bilde(row, url):
    val = url + r'\images' + "\\" + row['Borhull'] + '.png'
    arcpy.AddMessage(val)
    return val

In [15]:
def df_from_geosuite(xl_file: str, pdf_folder: str, crs) -> pd.DataFrame:
    cols = ['Borhull', 'X', 'Y', 'Z', 'Metode', 'Stopp', 'Løsm', 'Fjell']

    if xl_file.endswith('.xls'):
        df = pd.read_html(xl_file, decimal=',', thousands=None)[0][cols]
    else:
        df = pd.read_excel(xl_file, engine='openpyxl', usecols=cols)

    df['Borhull'] = df['Borhull'].astype("string")
    arcpy.AddMessage(df.dtypes)
    df['Tolket'] = df.apply(tolket, axis=1)
    df['Metode'] = df['Metode'].apply(remove_word, word='Tolk')
    df['Bergkote'] = df.apply(bergkote, axis=1)
    df['kommentar'] = df.apply(z_kom, axis=1)
    df['Bilde'] = df.apply(bilde, url=pdf_folder, axis=1)
    df.columns = [x.lower() for x in df.columns]

    #print(df.head())

    # convert df to spatially enabled dataframe
    sedf = pd.DataFrame.spatial.from_xy(df, "y", "x", sr=crs)  
    # notice the X and Y positions
    return df #sedf

In [16]:
xl_file = r'\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\10245026-01\10245026-01-03_ARBEIDSOMRAADE\21_fagomraade\11_Geoteknikk\10245026-10-GEOSUITE\Supplerende GRUS 2023\Test2.xls'
pdf_folder = r'\\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\10245026-01\10245026-01-03_ARBEIDSOMRAADE\21_fagomraade\11_Geoteknikk\10245026-10-GEOSUITE\Supplerende GRUS 2023\AUTOGRAF.RIT'
#crs = 'PROJCS["ETRS_1989_NTM_Zone_11",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",100000.0],PARAMETER["False_Northing",1000000.0],PARAMETER["Central_Meridian",11.5],PARAMETER["Scale_Factor",1.0],PARAMETER["Latitude_Of_Origin",58.0],UNIT["Meter",1.0]],VERTCS["NN2000_height",VDATUM["Norway_Normal_Null_2000"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]];-5523200 -15433400 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'
crs = 5111
df = df_from_geosuite(xl_file, pdf_folder, crs)
dfseries = df.squeeze()

    borhull            x          y       z metode  stopp   løsm  fjell  \
0   SB-2001  1142289.501  79222.189  31.802  Total     93  36.41   0.00   
1   SB-2002  1142293.917  79478.880  25.428  Total     90  17.12    NaN   
2   SB-2003  1142273.448  79194.757  31.360  Total     90  57.40    NaN   
3  SB-2003A  1142273.448  79194.757  31.360  Total     94  32.28   3.01   
4   SB-2004  1142271.376  79579.766  34.980  Total     94   6.48   3.00   

  tolket bergkote kommentar                                              bilde  
0           -4.61            \\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...  
1               ~            \\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...  
2               ~            \\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...  
3   Tolk    -0.92            \\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...  
4   Tolk     28.5            \\nsv2-nasuni-02\fredrikstad\Prosjekt\O10245\1...  


In [17]:
fl_hf.edit_features(adds=df)

{'addResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': False,
   'error': {'code': 1500, 'description': 'Incorrect geometry type.'}}],
 'updateResults': [],
 'deleteResults': []}

In [66]:
featurecol = FeatureLayerCollection.fromitem(fs_hf)
featurecol.manager.overwrite(df)
#fl_hf.manager.overwrite(df)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [57]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
active_map = aprx.activeMap

OSError: CURRENT

In [ ]:

# For verktøy 
#service_url = input_lyr.connectionProperties["connection_info"]["url"]
#bp_url = service_url + "/" + input_lyr.connectionProperties["dataset"]
#bp_source = FeatureLayer(bp_url)

In [17]:
hf_ID = '65bd4723a3444a28adf20299799138c8'
fs_hf = pgis.content.get(hf_ID)
fl_hf = fs_hf.layers[0]
fl_features = fl_hf.query().features

In [120]:
edit_feature = [f for f in fl_features if f.attributes['OBJECTID'] == 272][0]
updated_feature = edit_feature
updated_feature.attributes['Layer'] = 'Oppdatert'
update_res = fl_hf.edit_features(updates=[updated_feature])

In [121]:
update_res

{'addResults': [],
 'updateResults': [{'objectId': 272,
   'uniqueId': 272,
   'globalId': None,
   'success': True}],
 'deleteResults': []}

In [83]:
# Mappe
pdf_folder = r'\\nsv2-nasuni-01\Prosjekt\O10244\10244558-01\10244558-01-03 ARBEIDSOMRAADE\10244558-01 RIG\10244558-01-04 TEGNINGER\Kritiske profiler etter GRUS\PDF'
def img_folder_items(pdf_folder):
    items = []
    for item in os.listdir(pdf_folder):
        if item.split('.')[-1] == 'pdf':
            profilnr = item.split('_')[-1].split('.')[0]
            sonenavn = item.split('_')[-2]
            sonedict = {
                'profilnr': profilnr,
                'sonenavn': sonenavn,
                'attachment': pdf_folder+r'\\'+item
                }
            items.append(sonedict)
    return items


In [84]:
items = img_folder_items(pdf_folder=pdf_folder)
items

[{'profilnr': '5',
  'sonenavn': 'Skjelmerud',
  'attachment': '\\\\nsv2-nasuni-01\\Prosjekt\\O10244\\10244558-01\\10244558-01-03 ARBEIDSOMRAADE\\10244558-01 RIG\\10244558-01-04 TEGNINGER\\Kritiske profiler etter GRUS\\PDF\\\\82_Skjelmerud_5.pdf'},
 {'profilnr': '6',
  'sonenavn': 'Brotnu',
  'attachment': '\\\\nsv2-nasuni-01\\Prosjekt\\O10244\\10244558-01\\10244558-01-03 ARBEIDSOMRAADE\\10244558-01 RIG\\10244558-01-04 TEGNINGER\\Kritiske profiler etter GRUS\\PDF\\\\81_Brotnu_6.pdf'},
 {'profilnr': '2',
  'sonenavn': 'Brotnu',
  'attachment': '\\\\nsv2-nasuni-01\\Prosjekt\\O10244\\10244558-01\\10244558-01-03 ARBEIDSOMRAADE\\10244558-01 RIG\\10244558-01-04 TEGNINGER\\Kritiske profiler etter GRUS\\PDF\\\\81_Brotnu_2.pdf'},
 {'profilnr': '1',
  'sonenavn': 'Brotnu',
  'attachment': '\\\\nsv2-nasuni-01\\Prosjekt\\O10244\\10244558-01\\10244558-01-03 ARBEIDSOMRAADE\\10244558-01 RIG\\10244558-01-04 TEGNINGER\\Kritiske profiler etter GRUS\\PDF\\\\81_Brotnu_1.pdf'},
 {'profilnr': '10',
  'sonen

In [61]:
edit_feature = [f for f in fl_features if f.attributes['Profil_Sone'] =='Skjelmerud' and f.attributes['Profil_Profilnummer'] =='5'][0]
edit_feature.attributes.get('OBJECTID')

272

In [85]:
profiler = img_folder_items(pdf_folder)
def get_objectid(fl_features, profiler):
    objectids = []
    for profil in profiler:
        nr, navn, attachment = profil.get('profilnr'), profil.get('sonenavn'), profil.get('attachment')
        print(attachment)
        edit_feature = [f for f in fl_features if f.attributes['Profil_Sone'] ==navn and f.attributes['Profil_Profilnummer'] == nr][0]
        print(edit_feature.attributes.get('OBJECTID'))
        objectids.append(edit_feature.attributes.get('OBJECTID'))
    return objectids

objectids = get_objectid(fl_features, profiler)

\\nsv2-nasuni-01\Prosjekt\O10244\10244558-01\10244558-01-03 ARBEIDSOMRAADE\10244558-01 RIG\10244558-01-04 TEGNINGER\Kritiske profiler etter GRUS\PDF\\82_Skjelmerud_5.pdf
272
\\nsv2-nasuni-01\Prosjekt\O10244\10244558-01\10244558-01-03 ARBEIDSOMRAADE\10244558-01 RIG\10244558-01-04 TEGNINGER\Kritiske profiler etter GRUS\PDF\\81_Brotnu_6.pdf
99
\\nsv2-nasuni-01\Prosjekt\O10244\10244558-01\10244558-01-03 ARBEIDSOMRAADE\10244558-01 RIG\10244558-01-04 TEGNINGER\Kritiske profiler etter GRUS\PDF\\81_Brotnu_2.pdf
102
\\nsv2-nasuni-01\Prosjekt\O10244\10244558-01\10244558-01-03 ARBEIDSOMRAADE\10244558-01 RIG\10244558-01-04 TEGNINGER\Kritiske profiler etter GRUS\PDF\\81_Brotnu_1.pdf
246
\\nsv2-nasuni-01\Prosjekt\O10244\10244558-01\10244558-01-03 ARBEIDSOMRAADE\10244558-01 RIG\10244558-01-04 TEGNINGER\Kritiske profiler etter GRUS\PDF\\82_Skjelmerud_10.pdf
271


In [103]:
#fl_hf.properties.fields
fields = [(f.name, f.type) for f in fl_hf.properties.fields]



[{
   "name": "OBJECTID",
   "type": "esriFieldTypeOID",
   "alias": "OBJECTID",
   "sqlType": "sqlTypeOther",
   "nullable": false,
   "editable": false,
   "domain": null,
   "defaultValue": null
 },
 {
   "name": "Entity",
   "type": "esriFieldTypeString",
   "alias": "Entity",
   "sqlType": "sqlTypeOther",
   "length": 16,
   "nullable": true,
   "editable": true,
   "domain": null,
   "defaultValue": null
 },
 {
   "name": "Handle",
   "type": "esriFieldTypeString",
   "alias": "Handle",
   "sqlType": "sqlTypeOther",
   "length": 16,
   "nullable": true,
   "editable": true,
   "domain": null,
   "defaultValue": null
 },
 {
   "name": "Layer",
   "type": "esriFieldTypeString",
   "alias": "Layer",
   "sqlType": "sqlTypeOther",
   "length": 255,
   "nullable": true,
   "editable": true,
   "domain": null,
   "defaultValue": null
 },
 {
   "name": "LyrFrzn",
   "type": "esriFieldTypeSmallInteger",
   "alias": "LyrFrzn",
   "sqlType": "sqlTypeOther",
   "nullable": true,
   "editable

In [94]:
fl_hf.attachments.get_list(272)
    


[{'id': 244,
  'parentObjectId': 272,
  'name': 'Skjelmerud_5.png',
  'contentType': 'image/png',
  'size': 38059,
  'keywords': '',
  'exifInfo': None}]

In [ ]:
def update_attachments(fl):
    fl_features = fl.query().features
    arcpy.ResetProgressor()
    arcpy.SetProgressor('step', 'Processing features...', 0, len(fl_features), 1)
    a_counter = 0
    for feature in fl_features:
        oid = feature.attributes["objectid"]
        existing_attachments = fl.attachments.get_list(oid=oid)
        if existing_attachments:
            for a in existing_attachments:
                a_id = a['id']
                a_counter += 1
                fl.attachments.delete(oid,a_id)
                arcpy.AddMessage(f'Removing attachement {a["name"]}')
                # Update the progressor label
                arcpy.SetProgressorLabel(f"Processing {oid}, {a_id}...")
                # Update the progressor position
                arcpy.SetProgressorPosition()
        else:
             arcpy.AddMessage(f'No attachement {oid}')
    arcpy.AddMessage(f'Attachments deleted: {a_counter}')



In [104]:
fl_hf.connectionProperties["connection_info"]["url"]  

AttributeError: 'FeatureLayer' object has no attribute 'connectionProperties'